In [2]:
import os
from PIL import Image
from tqdm import tqdm
from transformers import InstructBlipForConditionalGeneration, InstructBlipProcessor
import torch

d:\PyCharm_Projects\Scraping Dataset\generating-annotations\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from config_file import config

In [4]:
# Путь к папке с изображениями
images_folder = config.IMAGES_PATH
output_file = config.DATA_PATH / "annotations.txt"

# ⚙️ Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
output_file = "annotations.txt"

In [ ]:
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-flan-t5-xl")
model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-flan-t5-xl",
    torch_dtype=torch.float16,  # 💡 экономим память
).to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
d:\PyCharm_Projects\Scraping Dataset\generating-annotations\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\badigadii\.cache\huggingface\hub\models--Salesforce--instructblip-flan-t5-xl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Win

In [ ]:
def generate_caption(image_path: str, prompt: str = "") -> str:
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        print(f"Ошибка при обработке {image_path}: {e}")
        return ""

In [23]:
Image.open(image_path).show()

In [ ]:
image_files = os.listdir(images_folder)
image_path = images_folder / image_files[0]
generate_caption(
    image_path,
    "Describe this image in rich detail, including objects, actions, background, and style",
)

"game screenshot of the day - the witch ' s house"

In [ ]:
# 📷 Обработка изображений
image_files = os.listdir(images_folder)
annotations = []

for img_file in tqdm(image_files, desc="Генерация аннотаций"):
    image_path = images_folder / img_file
    caption = generate_caption(image_path)
    if caption:
        annotations.append(f"{img_file}\t{caption}")

# 💾 Сохранение в файл
with open(output_file, "w", encoding="utf-8") as f:
    for line in annotations:
        f.write(line + "\n")

print(f"✅ Аннотации сохранены в {output_file}")

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'images'